In [2]:
#import beautifulsoup
from bs4 import BeautifulSoup
import os
import pandas as pd

In [3]:
cd c:\\Users\\drake\\Documents\\WGU\\Capstone\\AnalyticsCapstone\\

c:\Users\drake\Documents\WGU\Capstone\AnalyticsCapstone


In [4]:
files = []
path = 'data/raw'

if not os.path.exists(path):
    print(f"Directory '{path}' does not exist. Please run load_data.py first to download the files.")

for filename in os.listdir(path):
    full_path = os.path.join(path, filename)
    if os.path.isfile(full_path):
        files.append(full_path)

print(f"Found {len(files)} files in {path}")
for file in files:
    print(f"  - {file}")

Found 1175 files in data/raw
  - data/raw\Aaron Ashby.html
  - data/raw\Aaron Bummer.html
  - data/raw\Aaron Civale.html
  - data/raw\Aaron Judge.html
  - data/raw\Aaron Nola.html
  - data/raw\Aaron Schunk.html
  - data/raw\Abner Uribe.html
  - data/raw\Abraham Toro.html
  - data/raw\Adael Amador.html
  - data/raw\Adam Frazier.html
  - data/raw\Adam Mazur.html
  - data/raw\Adam Ottavino.html
  - data/raw\Addison Barger.html
  - data/raw\Adley Rutschman.html
  - data/raw\Adrian Houser.html
  - data/raw\Adrian Morejon.html
  - data/raw\AJ Blubaugh.html
  - data/raw\AJ Smith-Shawver.html
  - data/raw\Akil Baddoo.html
  - data/raw\Alan Rangel.html
  - data/raw\Alan Roden.html
  - data/raw\Alan Trejo.html
  - data/raw\Alec Bohm.html
  - data/raw\Alec Burleson.html
  - data/raw\Alejandro Kirk.html
  - data/raw\Alejandro Osuna.html
  - data/raw\Alek Jacob.html
  - data/raw\Alek Thomas.html
  - data/raw\Alex Bregman.html
  - data/raw\Alex Jackson.html
  - data/raw\Alex Verdugo.html
  - data/ra

In [8]:
#For each file, parse the html for the relevant stats and add it to the correct dataframe. Pitchers and batters have different stats, so they will be seperated.
batters_df = pd.DataFrame()
pitchers_df = pd.DataFrame()
num_files = len(files)
current_file = 0
for file in files[0:100]:
    current_file += 1
    print(f"Processing file {current_file} of {num_files}: {file}")
    with open(file, "r", encoding="utf-8") as f:
        html_content = f.read()

    player_name = file.split(".html")[0].split("\\")[-1]
    
    # Parse the HTML
    soup = BeautifulSoup(html_content, "html.parser")

    # Find the player's salary
    meta = soup.find(id="meta")
    for p in meta.find_all("p"):
        if "Contract Status" in p.text:
            # Extract salary value using regex
            import re
            contract_text = p.text
            # Look for salary pattern like $20.5M or $15.2M
            salary_match = re.search(r'\$(\d+\.?\d*[MBK]?)', contract_text)
            if salary_match:
                salary = salary_match.group(1)  # Get just the value without the $
                print(f"Salary: ${salary}")
            else:
                print("No salary found in contract status")

        # Find the table wrapper for pitching
    div = soup.find(id="div_players_value_pitching")
    if div:
        row = div.find(id="players_value_pitching.2024")
        if row:
            stats_dict = {}
            stats_dict['fullName'] = player_name
            year = row.find("th", {"data-stat": "year_id"}).text.strip()
            stats_dict["year"] = year
            for cell in row.find_all("td"):
                stat_name = cell.get("data-stat")
                stat_value = cell.text.strip()
                stats_dict[stat_name] = stat_value
                stats_dict['salary'] = salary
            df = pd.DataFrame([stats_dict])
            pitchers_df = pd.concat([pitchers_df, df], ignore_index=True)
        else:
            print("2025 pitching row not found.")
    else:
        print("Pitching table not found.")
    
    # Now handle batting
    div = soup.find(id="div_players_standard_batting")
    if div:
        row = div.find(id="players_standard_batting.2024")
        if row:
            stats_dict = {}
            stats_dict['fullName'] = player_name
            year = row.find("th", {"data-stat": "year_id"}).text.strip()
            stats_dict["year"] = year
            for cell in row.find_all("td"):
                stat_name = cell.get("data-stat")
                stat_value = cell.text.strip()
                stats_dict[stat_name] = stat_value
                stats_dict['salary'] = salary
            df = pd.DataFrame([stats_dict])
            batters_df = pd.concat([batters_df, df], ignore_index=True)
        else:
            print("2025 batting row not found.")
    else:
        print("Batting table not found.")

# return the dataframes

Processing file 1 of 1175: data/raw\Aaron Ashby.html
Salary: $20.5M
Batting table not found.
Processing file 2 of 1175: data/raw\Aaron Bummer.html
Salary: $13M
Batting table not found.
Processing file 3 of 1175: data/raw\Aaron Civale.html
Salary: $8M
Batting table not found.
Processing file 4 of 1175: data/raw\Aaron Judge.html
Salary: $360M
Pitching table not found.
Processing file 5 of 1175: data/raw\Aaron Nola.html
Salary: $172M
Batting table not found.
Processing file 6 of 1175: data/raw\Aaron Schunk.html
No salary found in contract status
Pitching table not found.
Processing file 7 of 1175: data/raw\Abner Uribe.html
No salary found in contract status
Batting table not found.
Processing file 8 of 1175: data/raw\Abraham Toro.html
No salary found in contract status
Pitching table not found.
Processing file 9 of 1175: data/raw\Adael Amador.html
No salary found in contract status
Pitching table not found.
Processing file 10 of 1175: data/raw\Adam Frazier.html
Salary: $1.53M
Pitching tab

In [ ]:
#Create a database to store the data
import sqlite3
import os

# Create database directory if it doesn't exist
db_dir = 'data/processed'
os.makedirs(db_dir, exist_ok=True)

# Create database connection
db_path = os.path.join(db_dir, 'baseball_stats.db')
conn = sqlite3.connect(db_path)

# Save pitchers dataframe to database
if not pitchers_df.empty:
    pitchers_df.to_sql('pitchers', conn, if_exists='replace', index=False)
    print(f"Saved {len(pitchers_df)} pitcher records to database")

# Save batters dataframe to database  
if not batters_df.empty:
    batters_df.to_sql('batters', conn, if_exists='replace', index=False)
    print(f"Saved {len(batters_df)} batter records to database")

# Close the connection
conn.close()
print(f"Database saved to: {db_path}")


In [ ]:
#Query the database for the data to make sure it worked

conn = sqlite3.connect('data/processed/baseball_stats.db')
batters_df = pd.read_sql_query("SELECT * FROM batters", conn)
print(f"Retrieved {len(batters_df)} batter records from database")
print(batters_df.head())


In [ ]:
# For the pitchers
conn = sqlite3.connect('data/processed/baseball_stats.db')
batters_df = pd.read_sql_query("SELECT * FROM pitchers", conn)
print(f"Retrieved {len(batters_df)} batter records from database")
print(batters_df.head())
